# Logistic Regression

Why do we call it a 'regression' when this is actually a classifier? The idea is that the regression doesnt output a class label, but probabilities indicating how likely the output has a label, and is good at **binary classification** based on features.

In the case of an image of some animal, which could either be a cat or not a cat. Say: $cat = 1, !cat = 0$.

Logistic regression works on binary classification, and produces a value $p\in [0,1]$

---

Say we have $n$ features in a feature vector $\mathbf{X}$, so each feature is $X_j^{(i)}$, and we have $m$ instances.

We take these $\mathbf{X}^{(i)}$ and a parameter matrix (weights and biases, ordinary linear regression) $\Theta\rightarrow \Theta^T\mathbf{X}^{(i)}=\mathbf{Z}^{(i)}$, a set of outputs - the logits.

For each feature, we have $n$ parameters. This is implied through linear algebra. Our goal is to find $\Theta$ using gradient descent and optimisation.

---

We need to find a function with a codomain of $f(x)\in(0,1)$. This is the sigmoid function.

$\phi(z) = \frac 1{1 + e^{-z}}$

This allows us to get to this:

$h_\Theta ^{(i)}\left( \mathbf{X}^{(i)} \right) = \phi\left(\mathbf{Z}^{(i)}\right)$

Where $h$ is the estimated probability. We need a function we can optimise now, and we think about this through a **likelihood** approach.

$\mathcal{L}\left(\Theta\right)=\prod_{i=1}^m \left[ h_\Theta ^{(i)}\left( \mathbf{X}^{(i)} \right) \right]^{y^{(i)}}\left[ 1-h_\Theta ^{(i)}\left( \mathbf{X}^{(i)} \right) \right]^{1-y^{(i)}}$

In [1]:
# say that the sigmoid function produced 0.999, and it is correct (ie. y=1)

print(0.999**1 * (1-0.999)**0)

0.999


In [2]:
# what about if sigmoid produces 0.999 but y=0?

print(0.999**0 * (1-0.999)**1)

0.0010000000000000009


## Loss function

If we get things right, we get numbers close to the sigmoid, and if we are wrong, the likelihood gets reduced.

There are a few faults though.
1. we need to negate the likelihood so that we can perform gradient descent, we want a loss function. So, we are going to minimise $\mathcal{L}^{-1}$
2. its numerically unstable, so we want to take $\ln$ too. $\rightarrow \ln{\mathcal{L}}$
3. We also want to divide by the number of instances, since logging the likelihood will turn it into a sum. $\rightarrow \frac1m\times\cdots$

$\Longrightarrow \ln{\mathcal{L}}=\sum_{i=1}^m {y^{(i)}\ln{h_\Theta (\mathbf{x}^{(i)})} + (1-y^{(i)})\ln{\left(1-h_\Theta (\mathbf{x}^{(i)})\right)}}$ is the log-likelihood. Applying the next two...

$
\Longrightarrow \mathcal{J}(\Theta)=-\frac1m \sum_{i=1}^m \left[{y^{(i)}\ln{h_\Theta (\mathbf{x}^{(i)})} + (1-y^{(i)})\ln{\left(1-h_\Theta (\mathbf{x}^{(i)})\right)} }\right]
$

This is called the "cross-entropy **loss function**". It's the negative average log likelihood, and it is also what we want to minimise when performing gradient descent.

---

## Gradient Descent

We have our parameters, $\Theta$. For each $\Theta_j$, how much does it have to change for the answer to be correct? How do we adjust the parameters to minimise the loss function?

We have $n$ features, so $n$ parameters, one for each feature.

$
\frac{\partial (\ln{\mathcal{L}})}{\partial \Theta_j} = \frac{\partial (\ln{\mathcal{L}})}{\partial h}\frac{\partial h}{\partial z}\frac{\partial z}{\partial \Theta_j}
$

The individual factors can be simplified, and was done on paper.

$
\longrightarrow \frac{\partial (\ln{\mathcal{L}})}{\partial \Theta_j} = \left[ \frac yh - \frac{1-y}{1-h} \right]\left[ h(1-h) \right]\mathbf{x}_j^{(i)} = (y-h)\mathbf{x}_j^{(i)}
$

$
\Longrightarrow \frac{\partial \mathcal{J}}{\partial \Theta_j} = -\frac{1}m (y^{(i)}-h)\mathbf{x}_j^{(i)}
$

We are interested in the vector form of this.

$
\nabla_\Theta \mathcal{J}(\Theta) = \frac1m \mathbf{X}^T\left( \mathbf{y}-h(\Theta X)\right)
$

With a learning rate $\alpha$, we take $\Theta\rightarrow\Theta-\alpha\cdot\nabla_\Theta \mathcal{J}(\Theta)$ in order to minimise our loss function.

In [3]:
# implement the important functions
import numpy as np

def sigmoid(z) -> float: # the sigmoid function
    return 1 / (1 + np.exp(-z))

def gradient(Theta, X, y) -> np.ndarray: # calculate the gradient for gradient descent
    return (X.T @ (sigmoid(X @ Theta) - y)) / y.size

def gradient_descent(X, y, alpha=0.1, num_iterations=200, tol=1e-7) -> np.ndarray:
    # appending a column of ones for each feature
    # note: this is, in effect, the "intercept" bias, much like in the linear regression algorithm
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]
    Theta = np.zeros(X_bias.shape[1])

    for i in range(num_iterations):
        grad = gradient(Theta, X_bias, y)
        Theta -= alpha * grad

        # check for tolerance
        if np.linalg.norm(grad) < tol:
            print("Tolerance hit, stopping gradient descent")
            break

    return Theta

## Predictions

Create functions to predict based on data

In [ ]:
def predict_probs(X, Theta): # no need to specify type, already in the sigmoid function
    X_bias = np.c_[np.ones((X.shape[0], 1)), X]

    return sigmoid(X_bias @ Theta)

def predict(X, Theta, threshold=0.5): # here, type is asserted in the "astype" method
    return (predict_probs(X, Theta) >= threshold).astype(int)

## Performance evaluation

Use scikit-learn's breast cancer dataset

In [ ]:
from sklearn.datasets import load_breast_cancer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score # just a simple evaluation

# Random seed for reproducibility
R = 260725 # date today is 26/7/25, just a fun random seed

# simple preprocessing, the data is nice
X, y = load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=R)

scaler = StandardScaler()

# scale the data
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

Theta_hat = gradient_descent(X_train, y_train, alpha=0.1, num_iterations=400)

y_pred_train = predict(X_train, Theta_hat)
y_pred_test = predict(X_test, Theta_hat)

training_accuracy = accuracy_score(y_train, y_pred_train)
testing_accuracy = accuracy_score(y_test, y_pred_test)

print("Training accuracy:", training_accuracy, "\n", "Testing accuracy:", testing_accuracy)

Training accuracy: 0.9846153846153847 
 Testing accuracy: 0.9736842105263158


Some very promising results. Here is where the code-along ends.